<a href="https://colab.research.google.com/github/zzhining/stock_market_analysis/blob/main/4%EC%A3%BC%EC%B0%A8/%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%88%98%EC%A7%911.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 구글 HTML 읽어오기

In [ ]:
import requests

url = 'https://www.google.com/'
reponse = requests.get(url)

In [ ]:
reponse.status_code

In [ ]:
reponse.text

In [ ]:
url = 'https://search.naver.com/search.naver?query=%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D'
response = requests.get(url)
response.text

# 이미지 가져오기

In [ ]:
url = 'https://www.google.com/images/branding/googlelogo/1x/googlelogo_light_color_272x92dp.png'

response = requests.get(url)

In [ ]:
response.status_code

In [ ]:
from PIL import Image
from io import BytesIO

Image.open(BytesIO(response.content))

In [ ]:
import os
name = os.path.basename(url)

In [ ]:
os.getcwd() #현재 작업 경로

In [ ]:
folder = f'{os.getcwd()}/image'

In [ ]:
# folder 경로가 없다면 folder 경로를 생성하기
if not os.path.isdir(folder):
    os.mkdir(folder)

In [ ]:
image_path = os.path.join(folder, name)

In [ ]:
image = open(image_path, 'wb') #image_path를 wb 모드로 오픈

In [ ]:
image.write(response.content) # response.content를 image 객체에 쓰기

In [ ]:
image.close()

# 블로그 실습

In [ ]:
!curl -L -O https://raw.githubusercontent.com/zzhining/python_data_basic/main/5%EC%9E%A5/index.html

In [ ]:
from bs4 import BeautifulSoup

# requests 대신에 index.html 파일을 직접 읽어오기
filename = "index.html"
html = ""
with open (filename, 'r', encoding='UTF-8') as file:  
    for line in file:               
        html += line
html

In [ ]:
soup = BeautifulSoup(html, 'lxml')

In [ ]:
post_list = soup.find_all('div', {'class': "post-preview"})

In [ ]:
title_list = []
subtitle_list= []
date_list = []

for post in post_list:
    title = post.find('h2', {'class':"post-title"}).text.strip()
    subtitle = post.find('h3').text.strip()
    date = post.find('p').text.strip()
    title_list.append(title)
    subtitle_list.append(subtitle)
    date_list.append(date)

In [ ]:
import pandas as pd
df = pd.DataFrame({'title': title_list, 'subtitle': subtitle_list, 'date': date_list})
df

# 신문기사 스크랩

In [ ]:
from bs4 import BeautifulSoup
import requests

url = 'https://finance.naver.com/news/news_read.naver?article_id=0004876048&office_id=008&mode=mainnews&type=&date=2023-04-17&page=1'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')
soup.find('div', {'class': 'articleCont'}).text

# 쇼핑몰 데이터

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# http://www.neweracapkorea.com/shop/shopbrand.html?xcode=031&mcode=002&type=Y&gf_ref=Yz1vU0FlS3M=
base_url = "http://www.neweracapkorea.com"
cap_total_url = "/shop/shopbrand.html?xcode=031&mcode=002&type=Y&gf_ref=Yz1vU0FlS3M="
base_url + cap_total_url

In [ ]:
# get html
response = requests.get(base_url+cap_total_url)

# BeautifulSoup을 활용하여 데이터 파싱
soup = BeautifulSoup(response.content, "lxml")

name_list = []
price_list = []
url_list = []

cap_info = soup.findAll('div', {'class':'tb-center'})

for cap in cap_info:
    name = cap.find('li', {'class':'dsc'}).text
    price = cap.find('li', {'class':'price'}).text
    url = cap.find('a').get('href')
    # print("이름: {}, 가격: {}".format(name, price))
    name_list.append(name)
    price_list.append(price)
    url_list.append(url)
df = pd.DataFrame({"이름": name_list, "가격" : price_list, "url": url_list})
df.head()

In [ ]:
cap_detail_df = pd.DataFrame()

for i in range(len(df)):
    print(i , end='... ')
    # (1)df에 저장된 url 읽어오기
    url = df['url'][i]
    response = requests.get(base_url + url)
    soup = BeautifulSoup(response.content, "lxml")
    
    # (2)상세페이지의 정보 파싱
    cap_name = soup.find('h3', {'class':'tit-prd'}).text.strip()
    cap_info = soup.find('tbody')
    features = cap_info.findAll('div', {'class':'tb-left'})
    
    fic_list = []
    for f in features:
        fic_list.append(f.text.strip())
    
    # (3)파싱한 정보를 사용하여 딕셔너리 생성
    cap_dict = {'제품명': cap_name}

    if len(fic_list) % 2 != 0:
        fic_list = fic_list[1:] 

    for i in range(len(fic_list)):
        if i % 2 == 0:
            if fic_list[i] != '':
                cap_dict.update({fic_list[i]: fic_list[i+1]})
    
    # (4)생성한 딕셔너리로 데이터프레임 생성
    temp_df = pd.DataFrame(data = cap_dict,  index = [0])

    # (5) 데이터프레임 추가
    if len(cap_detail_df) == 0:
        cap_detail_df = temp_df
    else:
        cap_detail_df = cap_detail_df.append(temp_df, ignore_index = True)

cap_detail_df.head()

In [ ]:
base_url = "http://www.neweracapkorea.com"

name_list = []
price_list = []
url_list = []
index = 1

while True: 
    try :
        print('{} 페이지 파싱.....'.format(index))
        page_url = f"/shop/shopbrand.html?type=Y&xcode=031&mcode=002&sort=&page={index}"
        response = requests.get(base_url+page_url)

        # BeautifulSoup을 활용하여 데이터 파싱
        soup = BeautifulSoup(response.content, "lxml")
        cap_info = soup.findAll('div', {'class':'tb-center'})

        if len(cap_info) == 0 :
            print('끝')
            break
        for cap in cap_info:
            name = cap.find('li', {'class':'dsc'}).text
            price = cap.find('li', {'class':'price'})
            url = cap.find('a').get('href')
            #print("이름: {}, 가격: {}".format(name, price))
            name_list.append(name)
            
            if price != None :
                price_list.append(price.text)
            else :
                price_list.append('SOLD OUT')
            url_list.append(url)
        index = index + 1
    except:
        print("Unexpected error:", sys.exc_info()[0])
        break

df = pd.DataFrame({"이름": name_list, "가격" : price_list, "url": url_list})
df.tail()